In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from astromodal.config import load_config
from tqdm import tqdm
import polars as pl
import random

In [3]:
config = load_config("/home/schwarz/projetoFM/config.yaml")

print(config['datacubes_paths'])

/home/astrodados4/downloads/hypercube/datacube_*.parquet


In [4]:
datacube_paths = config['datacubes_paths']

from astromodal.datasets.datacubes import load_datacube_files

In [7]:
train_files, val_files = load_datacube_files(
    datacubes_paths = datacube_paths,
    train_val_split = 1,
    nfiles_subsample = 1,
    seed = 42
)

[info] - Found 2444 datacube files
[info] - Subsampled to 1 files
[info] - Training files: 1
[info] - Validation files: 0


In [8]:
columns = [
    "splus_cut_F378",
    "splus_cut_F395",
    "splus_cut_F410",
    "splus_cut_F430",
    "splus_cut_F515",
    "splus_cut_F660",
    "splus_cut_F861",
    "splus_cut_R",
    "splus_cut_I",
    "splus_cut_Z",
    "splus_cut_U",
    "splus_cut_G",
]

bands = ["F378", "F395", "F410", "F430", "F515", "F660", "F861", "R", "I", "Z", "U", "G"]
cutout_size = 96

In [9]:
import polars as pl
from tqdm import tqdm

train_df = None

for f in tqdm(train_files, desc="Loading train files"):
    df = pl.read_parquet(f, columns=columns, use_pyarrow=True)
    df = df.filter(pl.col(columns[0]).is_not_null())

    if df.height == 0:
        continue

    train_df = df if train_df is None else pl.concat([train_df, df], how="vertical", rechunk=False)

train_df = train_df.rechunk()

Loading train files: 100%|██████████| 1/1 [00:06<00:00,  6.26s/it]


In [10]:
import polars as pl
from tqdm import tqdm

val_df = None

for f in tqdm(val_files, desc="Loading val files"):
    df = pl.read_parquet(f, columns=columns, use_pyarrow=True)
    df = df.filter(pl.col(columns[0]).is_not_null())

    if df.height == 0:
        continue

    val_df = df if val_df is None else pl.concat([val_df, df], how="vertical", rechunk=False)

val_df = val_df.rechunk()

Loading val files: 0it [00:00, ?it/s]

AttributeError: 'NoneType' object has no attribute 'rechunk'

In [11]:
from astromodal.datasets.spluscuts import SplusCutoutsDataset

train_dataset = SplusCutoutsDataset(
    train_df,
    bands=bands,
    img_size=cutout_size,
    return_valid_mask=True,
)
val_dataset = SplusCutoutsDataset(
    val_df,
    bands=bands,
    img_size=cutout_size,
    return_valid_mask=True,
)

AttributeError: 'NoneType' object has no attribute 'height'

In [21]:
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
import logpool

def train_epoch(
    model,
    dataloader,
    optimizer,
    device,
) -> float:
    model.train()
    total_loss = 0.0
    n_batches = 0

    for x_norm, m_valid in tqdm(dataloader, desc="Training", leave=False):
        x_norm = x_norm.to(device, non_blocking=True).float()
        m_valid = m_valid.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        x_recon, _ = model(x_norm)

        mv = m_valid > 0.5
        if mv.any():
            loss = F.mse_loss(x_recon[mv], x_norm[mv])
        else:
            loss = x_recon.sum() * 0.0

        loss.backward()
        optimizer.step()

        total_loss += float(loss.detach().cpu())
        n_batches += 1

    return total_loss / max(n_batches, 1)


@torch.no_grad()
def validate(
    model,
    dataloader,
    device,
) -> float:
    model.eval()
    total_loss = 0.0
    n_batches = 0

    for x_norm, m_valid in tqdm(dataloader, desc="Validating", leave=False):
        x_norm = x_norm.to(device, non_blocking=True).float()
        m_valid = m_valid.to(device, non_blocking=True)

        x_recon, _ = model(x_norm)

        mv = m_valid > 0.5
        if mv.any():
            loss = F.mse_loss(x_recon[mv], x_norm[mv])
        else:
            loss = x_recon.sum() * 0.0

        total_loss += float(loss.detach().cpu())
        n_batches += 1

    return total_loss / max(n_batches, 1)

In [15]:
from pathlib import Path

batch_size = 1024
max_gpu_batch_size = 1024
num_epochs = 10
learning_rate = 1e-3
latent_dim = 2

model_output_path = Path(config['models_folder']) / "./autoencoder_model_silu.pth"

In [23]:
from torch.utils.data import DataLoader

from astromodal.models.autoencoder import AutoEncoder
from torch.amp import GradScaler


train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=max_gpu_batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)



In [24]:
model = AutoEncoder(
    in_channels = len(bands),
    latent_dim = latent_dim,
    use_skips=False
)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import logpool
import torch

model = model.to(device)

best_val_loss = float("inf")

history = {
    "epoch": [],
    "train_loss": [],
    "val_loss": [],
}

for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_loss = validate(model, val_loader, device)

    history["epoch"].append(epoch)
    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)

    logpool.info(
        f"Epoch {epoch:03d}: Train Loss = {train_loss:.6f}, Val Loss = {val_loss:.6f}"
    )

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "train_loss": train_loss,
                "val_loss": val_loss,
                "history": history,   # 👈 saved here
                "config": {
                    "in_channels": len(bands),
                    "out_channels": len(bands),
                    "latent_dim": latent_dim,
                },
            },
            model_output_path,
        )
        logpool.info(f"  ✓ Saved best model (val_loss: {val_loss:.6f})")

Training:   0%|          | 0/454 [00:00<?, ?it/s]

In [18]:
import torch
from astromodal.models.autoencoder import AutoEncoder

def load_autoencoder(ckpt_path: str, device: torch.device):
    ckpt = torch.load(ckpt_path, map_location=device)

    # Try to recover config from checkpoint; fallback to your known values
    in_channels = len(bands)
    latent_dim = 2

    model = AutoEncoder(
        in_channels=in_channels,
        latent_dim=latent_dim,
    ).to(device)

    # support both raw state_dict or wrapped checkpoint
    state = ckpt["model_state_dict"] if "model_state_dict" in ckpt else ckpt
    model.load_state_dict(state, strict=True)

    model.eval()
    return model, ckpt

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, ckpt = load_autoencoder(model_output_path, device)

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import torch

def percentile_stretch(img2d: np.ndarray, p_lo=1, p_hi=99):
    finite = np.isfinite(img2d)
    if not finite.any():
        return img2d, 0.0, 1.0
    vmin = np.percentile(img2d[finite], p_lo)
    vmax = np.percentile(img2d[finite], p_hi)
    if vmax <= vmin:
        vmax = vmin + 1e-6
    out = np.clip((img2d - vmin) / (vmax - vmin), 0, 1)
    return out, vmin, vmax

def symmetric_residual_stretch(res2d: np.ndarray, p_hi=99):
    """
    Map residuals to [0,1] using symmetric limits [-v, +v],
    where v = percentile(|res|, p_hi).
    Zero residual -> 0.5.
    """
    finite = np.isfinite(res2d)
    if not finite.any():
        v = 1.0
        out = np.zeros_like(res2d) + 0.5
        return out, -v, v

    absvals = np.abs(res2d[finite])
    v = np.percentile(absvals, p_hi)
    if v <= 0:
        v = 1e-6

    out = np.clip((res2d + v) / (2 * v), 0, 1)
    return out, -v, v

def plot_before_after_residual_12bands(
    x_in: torch.Tensor,        # (C,H,W)
    x_rec: torch.Tensor,       # (C,H,W)
    band_names=None,
    p_lo=1,
    p_hi=99,
    use_input_percentiles=True,
    residual_p_hi=99,
    suptitle=None,
):
    x_in = x_in.detach().float().cpu().numpy()
    x_rec = x_rec.detach().float().cpu().numpy()
    res = x_rec - x_in

    C, H, W = x_in.shape
    assert C == 12, f"Expected 12 bands, got {C}"

    if band_names is None:
        band_names = [f"b{i}" for i in range(C)]

    fig, axes = plt.subplots(3, C, figsize=(2.2*C, 6.6), constrained_layout=True)

    for c in range(C):
        # same scaling for input & recon
        ref = x_in[c] if use_input_percentiles else np.concatenate([x_in[c].ravel(), x_rec[c].ravel()])
        _, vmin, vmax = percentile_stretch(ref, p_lo, p_hi)

        def norm_with(v):
            if vmax <= vmin:
                return np.zeros_like(v)
            return np.clip((v - vmin) / (vmax - vmin), 0, 1)

        im_in = norm_with(x_in[c])
        im_rec = norm_with(x_rec[c])

        # residual scaling: symmetric around 0
        im_res, rmin, rmax = symmetric_residual_stretch(res[c], p_hi=residual_p_hi)

        axes[0, c].imshow(im_in, origin="lower")
        axes[0, c].set_title(str(band_names[c]))
        axes[1, c].imshow(im_rec, origin="lower")
        axes[2, c].imshow(im_res, origin="lower")

        axes[0, c].axis("off")
        axes[1, c].axis("off")
        axes[2, c].axis("off")

        # optional: show residual limits on bottom row
        # axes[2, c].set_title(f"Δ [{rmin:.2g},{rmax:.2g}]", fontsize=8)

    axes[0, 0].set_ylabel("Input", fontsize=12)
    axes[1, 0].set_ylabel("Recon", fontsize=12)
    axes[2, 0].set_ylabel("Residual", fontsize=12)

    if suptitle:
        fig.suptitle(suptitle, fontsize=14)
    return fig

In [21]:
from torch.utils.data import DataLoader

def save_reconstructions(
    model,
    dataloader,
    device,
    band_names=None,
    n_examples=3,
    p_lo=1,
    p_hi=99,
    residual_p_hi=99,
    output_dir=None,
):
    batch = next(iter(dataloader))
    x = batch[0] if isinstance(batch, (tuple, list)) else batch

    x = x.to(device)

    with torch.no_grad():
        x_rec, z = model(x)

    for i in range(min(n_examples, x.shape[0])):
        fig = plot_before_after_residual_12bands(
            x_in=x[i],
            x_rec=x_rec[i],
            band_names=band_names,
            p_lo=p_lo,
            p_hi=p_hi,
            residual_p_hi=residual_p_hi,
            use_input_percentiles=True,
            suptitle=f"Example {i} | z shape: {tuple(z.shape)}",
        )
        if output_dir is not None:
            output_path = output_dir / f"reconstruction_{i}.png"
            fig.savefig(output_path)
            plt.close(fig)

In [22]:
loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)

bands = ["F378","F395","F410","F430","F515","F660","F861","U","G","R","I","Z"]

outfolder = Path(config['models_folder']) / "plots" / "autoencoder_reconstructions"
outfolder.mkdir(parents=True, exist_ok=True)

save_reconstructions(
    model=model,
    dataloader=loader,
    device=device,
    band_names=bands,
    n_examples=10,
    p_lo=1,
    p_hi=99,
    output_dir=outfolder
)